In [1]:
import os
import pandas as pd
import numpy as np
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.bigquery import magics
BIGQUERY_PROJECT = 'ironhacks-data'
bigquery_client = bigquery.Client(project=BIGQUERY_PROJECT)

In [2]:
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.unemployment_data`
"""
# QUERY THE DATA ONCE
query_job = bigquery_client.query(query)
unemp_data = query_job.to_dataframe()
unemp_data.head(5)

,uu_id,timeperiod,week_number,countyfips,tract,tract_name,total_claims,edu_8th_or_less,edu_grades_9_11,edu_hs_grad_equiv,...,gender_female,gender_male,gender_na,race_amerindian,race_asian,race_black,race_noanswer,race_hawaiiannative,race_other,race_white
0,f013068de98db1470bd986137a0c6d23,20220416,16,18003,900,"Census Tract 9, Allen County, Indiana",22,0,<NA>,14,...,<NA>,<NA>,0,0,0,11,0,0,<NA>,<NA>
1,21957d5517323845818d87623589e1ba,20220319,12,18089,10400,"Census Tract 104, Lake County, Indiana",111,0,<NA>,108,...,76,35,0,0,0,<NA>,<NA>,0,0,0
2,6a5609f385912113b6f1014b958ed748,20220326,13,18089,11500,"Census Tract 115, Lake County, Indiana",39,0,<NA>,<NA>,...,<NA>,<NA>,0,0,0,39,0,0,0,0
3,46b2882ec4c373527ec33f7bd4f1388d,20220716,29,18089,20700,"Census Tract 207, Lake County, Indiana",14,0,<NA>,<NA>,...,<NA>,<NA>,0,0,<NA>,10,0,0,<NA>,<NA>
4,37495d17e82f7df326bfc2c4c090f7b7,20220409,15,18089,21900,"Census Tract 219, Lake County, Indiana",155,0,<NA>,69,...,90,65,0,0,0,135,<NA>,<NA>,<NA>,<NA>


In [3]:
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.wage_data`
"""
# QUERY THE DATA ONCE
query_job = bigquery_client.query(query)
wage_data = query_job.to_dataframe()
wage_data.head()

,uu_id,countyfips,tract,tract_name,average_wage
0,585f8731c2255d6b3f817a31180848b9,18177,200,"Census Tract 2, Wayne County, Indiana",6612.000000
1,8c9d2aa90948679972a9382aadcc6001,18177,900,"Census Tract 9, Wayne County, Indiana",9883.250000
2,0f3d45341a5b113b813ffb7be7f58bab,18183,50300,"Census Tract 503, Whitley County, Indiana",13992.250000
3,fb55464f8e34af6d750d06968bf719b8,18183,50400,"Census Tract 504, Whitley County, Indiana",13613.500000
4,983badfd7b568728e39a2344a9006078,18001,30200,"Census Tract 302, Adams County, Indiana",11816.666667


In [4]:
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.prediction_list`
"""
# QUERY THE DATA ONCE
query_job = bigquery_client.query(query)
pred_data = query_job.to_dataframe()
pred_data.head()

,uu_id,week_number
0,5bf51fc2e162d6faf9e3cf79e4198378,44
1,420b44cc7e3f55d738df565421e59941,44
2,e39c66ecceec76ee8f9f811fa4a2d246,44
3,a90462cd11ae4e43144239bf7c4828a4,44
4,8b20a6749088c7ff1237983076ebfeaa,44


In [5]:
unemp_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16833 entries, 0 to 16832
Data columns (total 25 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   uu_id                   16833 non-null  object
 1   timeperiod              16833 non-null  Int64 
 2   week_number             16833 non-null  Int64 
 3   countyfips              16833 non-null  Int64 
 4   tract                   16833 non-null  Int64 
 5   tract_name              16833 non-null  object
 6   total_claims            16833 non-null  Int64 
 7   edu_8th_or_less         13748 non-null  Int64 
 8   edu_grades_9_11         5942 non-null   Int64 
 9   edu_hs_grad_equiv       6036 non-null   Int64 
 10  edu_post_hs             3246 non-null   Int64 
 11  edu_unknown             12031 non-null  Int64 
 12  top_category_employer1  16833 non-null  object
 13  top_category_employer2  16833 non-null  object
 14  top_category_employer3  16833 non-null  object
 15  ge

In [6]:
unemp_data['top_category_employer2'] = unemp_data['top_category_employer2'].replace('N/A',np.NaN)
unemp_data['top_category_employer3'] = unemp_data['top_category_employer3'].replace('N/A',np.NaN)

In [7]:
unemp_data['race_asian'] = unemp_data.groupby('countyfips')['race_asian'].transform(lambda x: x.fillna(int(x.mean())))

In [8]:
unemp_data['race_amerindian'] = unemp_data.groupby('countyfips')['race_amerindian'].transform(lambda x: x.fillna(int(x.mean())))

In [9]:
unemp_data['race_black'] = unemp_data.groupby('countyfips')['race_black'].transform(lambda x: x.fillna(int(x.mean())))

In [10]:
unemp_data['race_hawaiiannative'] = unemp_data.groupby('countyfips')['race_hawaiiannative'].transform(lambda x: x.fillna(int(x.mean())))
unemp_data['race_other'] = unemp_data.groupby('countyfips')['race_other'].transform(lambda x: x.fillna(int(x.mean())))
unemp_data['race_white'] = unemp_data.groupby('countyfips')['race_white'].transform(lambda x: x.fillna(int(x.mean())))

In [11]:
unemp_data['race_noanswer'] = unemp_data['race_noanswer'].fillna(0)
unemp_data['gender_na'] = unemp_data['gender_na'].fillna(0)
unemp_data['edu_unknown'] = unemp_data['edu_unknown'].fillna(0)

In [12]:
unemp_data['edu_8th_or_less'] = unemp_data.groupby('countyfips')['edu_8th_or_less'].transform(lambda x: x.fillna(int(x.mean())))
unemp_data['edu_grades_9_11'] = unemp_data.groupby('countyfips')['edu_grades_9_11'].transform(lambda x: x.fillna(int(x.mean())))
unemp_data['edu_hs_grad_equiv'] = unemp_data.groupby('countyfips')['edu_hs_grad_equiv'].transform(lambda x: x.fillna(int(x.mean())))

In [13]:
unemp_data['edu_post_hs'] = unemp_data.groupby('countyfips')['edu_post_hs'].transform(lambda x: x.fillna(int(x.mean())))

TypeError: int() argument must be a string, a bytes-like object or a number, not 'NAType'

In [14]:
unemp_data['edu_post_hs'] = unemp_data.groupby('countyfips')['edu_post_hs'].transform(lambda x: x.fillna(int(x.mean()) if not x.isnull() else 0))

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [15]:
unemp_data['edu_post_hs'] = unemp_data.groupby('countyfips')['edu_post_hs'].transform(lambda x: x.fillna(int(x.mean()) if not x.mean().empty else 0))

AttributeError: 'numpy.float64' object has no attribute 'empty'

In [16]:
unemp_data['edu_post_hs'] = unemp_data.groupby('countyfips')['edu_post_hs'].transform(lambda x: x.fillna(int(x.mean()) if not x.mean().isnull() else 0))

AttributeError: 'numpy.float64' object has no attribute 'isnull'

In [17]:
import numpy as np
unemp_data['edu_post_hs'] = unemp_data['edu_post_hs'].replace(NA,np.NaN)
unemp_data['edu_post_hs'] = unemp_data.groupby('countyfips')['edu_post_hs'].transform(lambda x: x.fillna(int(x.mean())))

NameError: name 'NA' is not defined

In [18]:
import numpy as np
unemp_data['edu_post_hs'] = unemp_data['edu_post_hs'].replace(np.NaN,np.NaN)
unemp_data['edu_post_hs'] = unemp_data.groupby('countyfips')['edu_post_hs'].transform(lambda x: x.fillna(int(x.mean())))

TypeError: int() argument must be a string, a bytes-like object or a number, not 'NAType'

In [19]:
unemp_data['edu_post_hs']

0        <NA>
1        <NA>
2        <NA>
3        <NA>
4          83
         ... 
16828      12
16829    <NA>
16830    <NA>
16831    <NA>
16832    <NA>
Name: edu_post_hs, Length: 16833, dtype: Int64

In [20]:
import numpy as np
unemp_data['edu_post_hs'] = unemp_data['edu_post_hs'].replace('<NA>',np.NaN)
unemp_data['edu_post_hs'] = unemp_data.groupby('countyfips')['edu_post_hs'].transform(lambda x: x.fillna(int(x.mean())))

TypeError: int() argument must be a string, a bytes-like object or a number, not 'NAType'

In [21]:
unemp_data['edu_post_hs']

0        <NA>
1        <NA>
2        <NA>
3        <NA>
4          83
         ... 
16828      12
16829    <NA>
16830    <NA>
16831    <NA>
16832    <NA>
Name: edu_post_hs, Length: 16833, dtype: Int64

In [22]:
import numpy as np
unemp_data['edu_post_hs'] = unemp_data['edu_post_hs'].replace(<NA>,np.NaN)
unemp_data['edu_post_hs'] = unemp_data.groupby('countyfips')['edu_post_hs'].transform(lambda x: x.fillna(int(x.mean())))

SyntaxError: invalid syntax (1546246240.py, line 2)

In [23]:
unemp_data['edu_post_hs']

0        <NA>
1        <NA>
2        <NA>
3        <NA>
4          83
         ... 
16828      12
16829    <NA>
16830    <NA>
16831    <NA>
16832    <NA>
Name: edu_post_hs, Length: 16833, dtype: Int64

In [24]:
unemp_data['edu_post_hs'] = unemp_data.groupby('countyfips')['edu_post_hs']\
    .transform(lambda x: x.fillna(x.mean()))

TypeError: Invalid value '17.321100917431192' for dtype Int64

In [25]:
unemp_data['edu_post_hs'] = unemp_data.groupby('countyfips')['edu_post_hs']\
    .transform(lambda x: x.fillna(int(x.mean())))

TypeError: int() argument must be a string, a bytes-like object or a number, not 'NAType'

In [26]:
#unemp_data['gender_female'] = unemp_data.groupby('countyfips')['gender_female'].transform(lambda x: x.fillna(int(x.mean())))
unemp_data['gender_male'] = unemp_data.groupby('countyfips')['gender_male'].transform(lambda x: x.fillna(int(x.mean())))

TypeError: int() argument must be a string, a bytes-like object or a number, not 'NAType'

In [27]:
unemp_data.isna().sum()

uu_id                         0
timeperiod                    0
week_number                   0
countyfips                    0
tract                         0
tract_name                    0
total_claims                  0
edu_8th_or_less               0
edu_grades_9_11               0
edu_hs_grad_equiv             0
edu_post_hs               13587
edu_unknown                   0
top_category_employer1        0
top_category_employer2       49
top_category_employer3      274
gender_female             12998
gender_male               12902
gender_na                     0
race_amerindian               0
race_asian                    0
race_black                    0
race_noanswer                 0
race_hawaiiannative           0
race_other                    0
race_white                    0
dtype: int64

In [28]:
unemp_data['top_category_employer2'] = unemp_data.groupby('countyfips')['top_category_employer2'].transform(lambda x: x.fillna(int(x.mode()[0] if not x.mode().empty else 'unknown')))

ValueError: invalid literal for int() with base 10: '31-33'

In [29]:
unemp_data['top_category_employer2'] = unemp_data.groupby('countyfips')['top_category_employer2'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 'unknown'))

In [30]:
unemp_data['top_category_employer3'] = unemp_data.groupby('countyfips')['top_category_employer3'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 'unknown'))

In [31]:
unemp_data.isna().sum()

uu_id                         0
timeperiod                    0
week_number                   0
countyfips                    0
tract                         0
tract_name                    0
total_claims                  0
edu_8th_or_less               0
edu_grades_9_11               0
edu_hs_grad_equiv             0
edu_post_hs               13587
edu_unknown                   0
top_category_employer1        0
top_category_employer2        0
top_category_employer3        0
gender_female             12998
gender_male               12902
gender_na                     0
race_amerindian               0
race_asian                    0
race_black                    0
race_noanswer                 0
race_hawaiiannative           0
race_other                    0
race_white                    0
dtype: int64

In [32]:
unemp_data['edu_post_hs'] = unemp_data.groupby('countyfips')['edu_post_hs']\
    .transform(lambda x: x.fillna(int(x.mean()[0])))

IndexError: invalid index to scalar variable.

In [33]:
unemp_data['edu_post_hs'] = unemp_data.groupby('countyfips')['edu_post_hs']\
    .transform(lambda x: x.fillna(int(x.mean())[0]))

TypeError: 'int' object is not subscriptable

In [34]:
unemp_data['edu_post_hs'] = unemp_data.groupby('countyfips')['edu_post_hs'].fillna(0)

In [35]:
#unemp_data['gender_female'] = unemp_data.groupby('countyfips')['gender_female'].transform(lambda x: x.fillna(int(x.mean())))
unemp_data['gender_male'] = unemp_data['gender_male'].fillna(unemp_data.groupby('countyfips')['gender_male'].transform(mean))

NameError: name 'mean' is not defined

In [36]:
#unemp_data['gender_female'] = unemp_data.groupby('countyfips')['gender_female'].transform(lambda x: x.fillna(int(x.mean())))
unemp_data['gender_male'] = unemp_data['gender_male'].fillna(unemp_data.groupby('countyfips')['gender_male'].transform('mean'))

TypeError: cannot safely cast non-equivalent object to int64

In [37]:
unemp_data['gender_male'] = unemp_data['gender_male'].astype(float)

In [38]:
#unemp_data['gender_female'] = unemp_data.groupby('countyfips')['gender_female'].transform(lambda x: x.fillna(int(x.mean())))
unemp_data['gender_male'] = unemp_data['gender_male'].fillna(unemp_data.groupby('countyfips')['gender_male'].transform('mean'))

In [39]:
unemp_data['gender_female'] = unemp_data['gender_female'].fillna(unemp_data.groupby('countyfips')['gender_female'].transform('mean'))
unemp_data['edu_post_hs'] = unemp_data['edu_post_hs'].fillna(unemp_data.groupby('countyfips')['edu_post_hs'].transform('mean'))

TypeError: cannot safely cast non-equivalent object to int64

In [40]:
unemp_data['gender_male'] = unemp_data['gender_male'].astype(float)
unemp_data['gender_female'] = unemp_data['gender_female'].astype(float)
unemp_data['edu_post_hs'] = unemp_data['edu_post_hs'].astype(float)

In [41]:
unemp_data['gender_female'] = unemp_data['gender_female'].fillna(unemp_data.groupby('countyfips')['gender_female'].transform('mean'))
unemp_data['edu_post_hs'] = unemp_data['edu_post_hs'].fillna(unemp_data.groupby('countyfips')['edu_post_hs'].transform('mean'))